# Chapter 9  
## Metaprogramming

One of the most important mantras of software development is "don’t repeat yourself."  
That is, any time you are faced with a problem of creating highly repetitive code (or cutting and pasting source code), it often pays to look for a more elegant solution.  
In Python, such problems are often solved under the category of "metaprogramming."  
In a nutshell, metaprogramming is about creating functions and classes whose main goal is to manipulate code by modifying, generating, or wrapping existing code.  
The main features for this include decorators, class decorators, and metaclasses.  
However, a variety of other useful topics -- including signature objects, execution of code with exec(), and inspecting the internals of functions and classes -- enter the picture.  
The main purpose of this chapter is to explore various metaprogramming techniques and to give examples of how they can be used to customize the behavior of Python to your own whims.

## 9.1 Putting a Wrapper Around a Function

### Problem

You want to put a wrapper layer around a function that adds extra processing, such as logging, timing, and so forth.

### Solution

You can define a decorator function to wrap a function with extra code.

In [1]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports the execution time.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

Now let's make our decorator useful.

In [2]:
@timethis
def countdown(n):
    """
    Yes, just a countdown.
    """
    while n > 0:
        n -= 1

In [3]:
countdown(10000)

countdown 0.0006248950958251953


In [4]:
countdown(100000)

countdown 0.006066799163818359


### Discussion

A decorator is a function that accepts a function as input and returns a new function as output.  
Whenever you write code like the following:

In [5]:
@timethis
def countdown(n):
    '''do something'''

it's the same as if you had executed these separate steps:

In [6]:
def countdown(n):
    '''...'''
countdown = timethis(countdown)

Python's built-in decorators, like `@staticmethod`, `@classmethod`, and `@property` work in the same way.  
For example, the following two code fragments will give an equivalent result:

In [7]:
class A:
    @classmethod
    def method(cls):
        pass
    
class B:
    # Equivalent definition of a class method.
    def method(cls):
        pass
    method = classmethod(method)

The code inside a decorator typically involves creating a new function that accepts any arguments using `*args` and `**kwargs`, as shown with the `wrapper()` function in this recipe.  
Inside this function, you place a call to the original input function and return its result.  
However, you also place whatever extra code you want to add, like our timing function.  
The newly created function wrapper is returned as a result and takes the place of the original function.  
It’s critical to emphasize that decorators generally do not alter the calling signature or return value of the function being wrapped.  
The use of `*args` and `**kwargs` is there to make sure that any input arguments can be accepted.  
The return value of a decorator is almost always the result of calling `func(*args, **kwargs)`, where `func` is the original unwrapped function.  
When first learning about decorators, it is usually very easy to get started with some simple examples, such as the one shown.  
However, if you are going to write decorators for real, there are some subtle details to consider.  
For example, the use of the decorator `@wraps(func)` in the solution is an easy to forget but important technicality related to preserving function metadata, which is described in the next recipe.  
The next few recipes that follow fill in some details that will be important if you wish to write decorator functions of your own.

## 9.2. Preserving Function Metadata When Writing Decorators

### Problem

You've written a decorator, but when you apply it to a function, important metadata such as the name, docstring, annotations, and calling signature are lost.

### Solution

Whenever you define a decorator, you should always remember to apply the `@wraps` decorator from the `functools` library to the underlying wrapper function.

In [8]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports execution time.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

Let's use the decorator and examine the resulting function metadata:

In [9]:
@timethis
def countdown(n:int):
    """
    Countdown the integers.
    """
    while n > 0:
        n -= 1

In [10]:
countdown(10000)

countdown 0.0005581378936767578


In [11]:
countdown(100000)

countdown 0.006112098693847656


In [12]:
countdown.__name__

'countdown'

In [13]:
countdown.__doc__

'\n    Countdown the integers.\n    '

In [14]:
countdown.__annotations__

{'n': int}

### Discussion

Copying decorator metadata is an important part of writing decorators.  
If you forget to use `@wraps`, you’ll find that the decorated function loses all sorts of useful information.  
For instance, if omitted, the metadata in the last example would look like this:

An important feature of the `@wraps` decorator is that it makes the wrapped function available to you in the `__wrapped__` attribute.  
For example, if you want to access the wrapped function directly, you could do this:

The presence of the `__wrapped__` attribute also makes decorated functions properly expose the underlying signature of the wrapped function.

In [15]:
from inspect import signature
print(signature(countdown))

(n:int)


One common question that sometimes arises is how to make a decorator that directly copies the calling signature of the original function being wrapped (as opposed to using `*args` and `**kwargs`).  
In general, this is difficult to implement without resorting to some trick involving the generator of code strings and `exec()`.  
Frankly, you’re usually best off using `@wraps` and relying on the fact that the underlying function signature can be propagated by access to the underlying `__wrapped__` attribute.  
See Recipe 9.16 for more information about signatures.

## 9.3. Unwrapping a Decorator

### Problem

A decorator has been applied to a function, but you want to overrride it and gain access to the original unwrapped function.

### Solution

Assuming that the decorator has been implemented properly using `@wraps` (see Recipe 9.2), you can usually gain access to the original function by accessing the `__wrapped__` attribute.

### Discussion

Gaining direct access to the unwrapped function behind a decorator can be useful for debugging, introspection, and other operations involving functions.  
However, this recipe only works if the implementation of a decorator properly copies metadata using `@wraps` from the functools module or sets the `__wrapped__` attribute directly.  
If multiple decorators have been applied to a function, the behavior of accessing `__wrapped__` is currently undefined and should probably be avoided.  
In Python 3.3, it bypasses all of the layers.  
For example, suppose you have code like this:

In [16]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

In [17]:
@decorator1
@decorator2
def add(x, y):
    return x + y

Now let's call the decorated function and the original function through `__wrapped__`.

In [18]:
add(2, 3)

Decorator 1
Decorator 2


5

In [19]:
add.__wrapped__(2, 3)

Decorator 2


5

However, this behavior has been reported as a bug (see `http://bugs.python.org/issue17482`) and may be changed to expose the proper decorator chain in a future release.
Last, but not least, be aware that not all decorators utilize `@wraps`, and thus, they may not work as described.  
In particular, the built-in decorators `@staticmethod` and `@classmethod` create descriptor objects that don’t follow this convention (instead, they store the original function in a `__func__` attribute).  
Your mileage may vary.

## 9.4. Defining a Decorator That Takes Arguments

### Problem

You want to write a decorator function that takes arguments.

### Solution

Let’s illustrate the process of accepting arguments with an example.  
Suppose you want to write a decorator that adds logging to a function, but allows the user to specify the logging level and other details as arguments.  
Here is how you might define the decorator:

In [20]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    """
    Add logging to a function.
    If name and message aren't specified, they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.level(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

Test drive our code:

In [21]:
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

On first glance, the implementation looks tricky, but the idea is relatively simple.  
The outermost function `logged()` accepts the desired arguments and simply makes them available to the inner functions of the decorator.  
The inner function `decorate()` accepts a function and puts a wrapper around it as normal.  
The key part is that the wrapper is allowed to use the arguments passed to `logged()`.

### Discussion

Writing a decorator that takes arguments is tricky because of the underlying calling sequence involved.  
For example, if you have code like this:

The decoration process evaluates as follows:

Carefully observe that the result of `decorator(x, y, z)` must be a callable which, in turn, takes a function as input and wraps it.  
See Recipe 9.7 for another example of a decorator taking arguments.

## 9.5. Defining a Decorator with User Adjustable Attributes

### Problem

You want to write a decorator function that wraps a function, but has user adjustable attributes that can be used to control the behavior of the decorator at runtime.

### Solution

Here is a solution that expands on the last recipe by introducing accessor functions that change internal variables through the use of nonlocal variable declarations.  
The accessor functions are then attached to the wrapper function as function attributes.

In [22]:
from functools import wraps, partial
import logging

# Utility decorator to attach a function as an attribute of obj:
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func
def logged(level, name=None, message=None):
    """
    Add logging to a function.
    If name and message aren't specified, they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        
        # Attach setter functions:
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
            
        return wrapper
    return decorate

# Example use:
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

Here is an interactive session that shows the various attributes being changed after definition:

In [23]:
import logging
logging.basicConfig(level=logging.DEBUG)
add(2, 3)

DEBUG:__main__:add


5

In [24]:
# Change the log message:
add.set_message('Add called')
add(2, 3)

DEBUG:__main__:Add called


5

In [25]:
# Change the log level:
add.set_level(logging.WARNING)
add(2, 3)

5

### Discussion

The key to this recipe lies in the accessor functions, like `set_message()` and `set_level()`, that get attached to the wrapper as attributes.  
Each of these accessors allows internal parameters to be adjusted through the use of nonlocal assignments.  
An added feature of this recipe is that the accessor functions will propagate through multiple levels of decoration if all of your decorators use `@functools.wraps`.  
For example, suppose you introduced an additional decorator, such as the `@timethis` decorator from Recipe 9.2, and wrote code like this:

In [26]:
@timethis
@logged(logging.DEBUG)
def countdown(n):
    while n > 0:
        n -= 1

In [27]:
countdown(10000)

DEBUG:__main__:countdown


countdown 0.001432180404663086


In [28]:
countdown.set_level(logging.WARNING)
countdown.set_message("Ground control to Major Tom, prepare for blastoff")
countdown(10000)

countdown 0.002003192901611328


If the decorators are composed in the exact opposite order, everything still works exactly the same:

In [29]:
@logged(logging.DEBUG)
@timethis
def countdown(n):
    while n > 0:
        n -= 1

You can make additional modifications, such as writing accessor functions to return the values of varous settings:

Or an alternative:

One extremely subtle facet of this recipe is the choice to use accessor functions in the first place.  
For example, you might consider an alternative formulation solely based on direct access to function attributes like this:

This approach would work to a point, but only if it was the topmost decorator.  
If you had another decorator applied on top, such as the `@timethis` example, it would shadow the underlying attributes and make them unavailable for modification.  
The use of accessor functions avoids this limitation.  
The solution shown in this recipe might also be a possible alternative for decorators defined as classes, as shown in Recipe 9.9.

## 9.6. Defining a Decorator That Takes an Optional Argument